In [30]:
import csv
import json
import pandas as pd
from datetime import datetime
import time


vendorMaster = {}
with open('/Users/patelrudra/Documents/Medkart/wms/Contacts.csv', mode ='r')as file:
    masterFile = csv.reader(file)    
    for vendor in masterFile:
        vendorMaster[vendor[1]] = vendor[0]
        
vendorItem = {}
with open('/Users/patelrudra/Documents/BOOKS/Item.csv', mode ='r')as file1:
    masterFile = csv.reader(file1)    
    for vendor in masterFile:
        vendorItem[vendor[1]] = vendor[0]

vendorgst = {}
with open('/Users/patelrudra/Documents/BOOKS/gst.csv', mode ='r')as file2:
    masterFile = csv.reader(file2)    
    for vendor in masterFile:
        vendorgst[vendor[0]] = vendor[1]        
        
        
gstConfigurationData = {
    "0" : "1357403000000016427",
    "5" : "1357403000000016433",
    "12" : "1357403000000016439",
    "18" : "1357403000000016445",
    "28" : "1357403000000016451"
}
        
with open('/Users/patelrudra/Documents/Medkart/wms/credit note/newcreditnotejune.csv', mode ='r')as file:

    csvFile = csv.reader(file)
    next(csvFile)
    data_list = []
    null_voucher_dict = []
    data={}
    count=0
    Bill=True
    notItems=[]
    
    for entry in csvFile:
        if entry[4] == '':
            if entry[1] == 'Grand Total':
                continue;
            data1 = {
                'item_id': vendorItem[entry[1]],
                'rate':entry[10],
                "quantity":"1",
                "item_order":len(data_list)+1
             }
            
            data_list.append(data1)
        else:
            data['line_items'] = data_list
            null_voucher_dict.append(data)
            data_list=[]
            date_object = datetime.strptime(entry[0],"%d-%b-%y")
            formatted_date = date_object.strftime("%Y-%m-%d")
            data = {
                'date': formatted_date,
                'customer_id': vendorMaster[entry[1]],
                "creditnote_number" : entry[4],
#                 "gst_no":entry[5],
                 "is_draft":False,
                "is_inclusive_tax": False,
                "reference_number":entry[5],
                "adjustment": entry[17],
                'reference_invoice_type':vendorgst[entry[1]],
#                 'gst_treatment':"business_none",
                'gst_reason':"sales_return",
                "adjustment_description": "adjustment",
                
             }
    data['line_items'] = data_list
    null_voucher_dict.append(data)
            
    for data in null_voucher_dict:
        json_data = json.dumps(data, indent=4)
        with open("sample.json", "a") as outfile:
            outfile.write(json_data + '\n')  

In [31]:
null_voucher_dict.pop(0)
print(len(null_voucher_dict))
null_voucher_dict


283


[{'date': '2023-06-25',
  'customer_id': '1357403000004193657',
  'creditnote_number': '230301SR00000092',
  'is_draft': False,
  'is_inclusive_tax': False,
  'reference_number': '230301SC00005987',
  'adjustment': '-0.01',
  'reference_invoice_type': 'b2c_others',
  'gst_reason': 'sales_return',
  'adjustment_description': 'adjustment',
  'line_items': [{'item_id': '1357403000004360018',
    'rate': '21.43',
    'quantity': '1',
    'item_order': 1},
   {'item_id': '1357403000004360018',
    'rate': '40.18',
    'quantity': '1',
    'item_order': 2}]},
 {'date': '2023-06-25',
  'customer_id': '1357403000004192931',
  'creditnote_number': '230341SR00000033',
  'is_draft': False,
  'is_inclusive_tax': False,
  'reference_number': '230341SC00002159',
  'adjustment': '',
  'reference_invoice_type': 'b2c_others',
  'gst_reason': 'sales_return',
  'adjustment_description': 'adjustment',
  'line_items': [{'item_id': '1357403000004360420',
    'rate': '80.36',
    'quantity': '1',
    'item_o

In [32]:
import requests
import json

url = "https://books.zoho.in/api/v3/creditnotes?organization_id=60022053195"
headers = {
  'Content-Type': 'application/json',
  'Authorization': 'Zoho-oauthtoken 1000.cee2302644d625f600bab72c57e2b328.599d372d4bef9b83fa537fc598365e3f'
}
failedData = []
for data in null_voucher_dict:
    payload = json.dumps(data)


    response = requests.request("POST", url, headers=headers, data=payload)
    if response.status_code >= 200 and response.status_code < 300:  # Assuming a successful response has a status code of 200
        data = response.json()
        
    else:
        print(f"API request failed with status code {response.status_code}")
        response_content = response.content.decode('utf-8')
        failedData.append(data['creditnote_number'])
        print(response_content)
    time.sleep(6)    

print("__________________________________")
print("Finished")
print("__________________________________")
print("failed records bill number : ")
print(failedData)

__________________________________
Finished
__________________________________
failed records bill number : 
[]
